## Goroutines

- A goroutine is a lightweight thread managed by the Go runtime.
    ```go
    go f(x, y, z)
    ```

- e.g.
    ```go
    package main

    func main() {
        go func() {
            println("abc")
        }()

        func() {
            println("xyz")
        }()
    }
    ```

- output

    ```go
    xyz
    ```
    
- This has output only `xyz` because the `main` function executes the second function and exits even before the 1st function(go routine) starts/executes.

- e.g.

    ```go
    func main() {
        go func() {
            println("abc")
        }()

        go func() {
            println("xyz")
        }()
	
        time.Sleep(time.Second * 1)	
    }
    ```
    
- If the above program has no `Sleep` function call then there will be no output shown just because `main` will exit before others, introducing `time` helps.

- If `GOMAXPROCS` value is one and we have multiple go routines then all these go routines will be spawned as threads running concurrently, no parallelism, but if the `GOMAXPROS` > 1 then this means newer go routine threads will be allocated to other processor and will be run in parallel.

- e.g.

    ```go
    func main() {
    	//runtime.GOMAXPROCS(2)                          // Uncomment for 3rd output                   
        go func() {
            for i := 0; i < 5; i++ {
                println("abc")
                //time.Sleep(time.Millisecond * 10)      // Uncomment for 2nd and 3rd output
            }
        }()

        go func() {
            for i := 0; i < 5; i++ {
                println("xyz")
                //time.Sleep(time.Millisecond * 10)      // Uncomment for 2nd and 3rd output     
            }
        }()
        time.Sleep(time.Second * 1)
    }
    ```

-    output:
    ```bash
    abc                   ## This output is like this because each function is running
    abc                   ## concurrently so they execute comletely in very short amount
    abc                   ## of time, even if they are two threads
    abc
    abc
    xyz
    xyz
    xyz
    xyz
    xyz
    ```

-    output with `time.Sleep`
    ```bash
    abc               ## here we can see a proper multi-threaded output because each thread/routine
    xyz               ## sleeps for some time so the runtime runs other thread while one sleeps
    xyz
    abc
    abc
    xyz
    xyz
    abc
    abc
    xyz
    
    ```

- output with `time.Sleep` and `runtime.GOMAXPROCS`

    ```bash
    abc
    xyz           ## Here actual parallel processing has been done, so here output can be random
    abc           ## sometimes two things can be printed on the same line.
    xyz
    abc
    xyz
    xyz
    abc
    xyz
    abc
    ```

- Things that are done sequentially can be spawned in different routines.
- Scenario: Downloading some data from some website and then parsing it and showing the results, can be paralleled. Becasue most of the times, the network io is the blocking call, so the execution speeds can be increased by making it parallel.

## Channels

- Channels are a typed conduit through which you can send and receive values with the channel operator, `<-`.
    ```go
    ch <- v    // Send v to channel ch.
    v := <-ch  // Receive from ch, and
               // assign value to v.
    ```

- (The data flows in the direction of the arrow.)
- Like maps and slices, channels must be created before use:
    ```go
    ch := make(chan int)
    ```
    
- By default, sends and receives block until the other side is ready. This allows goroutines to synchronize without explicit locks or condition variables.

- When reading from a channel which has no data, it gives an error 
    ```bash
    fatal error: all goroutines are asleep -deadlock!
    ```
- Since no goroutine is writing to it and this is the only goroutine waiting to read, the runtime recognizes this is an errorsome situation and exits out.

## Buffered Channels

- Channels can be buffered. Provide the buffer length as the second argument to make to initialize a buffered channel:
    ```go
    ch := make(chan int, 100)
    ```

- Sends to a buffered channel block only when the buffer is full. Receives block when the buffer is empty.
- A sender can close a channel to indicate that no more values will be sent. Receivers can test whether a channel has been closed by assigning a second parameter to the receive expression: after:
    ```go
    v, ok := <-ch
    ```
- `ok` is `false` if there are no more values to receive and the channel is closed.
- Normally channels don't have storage or buffer, so when using a normal channel whenever you write to it, it needs a receiver ready parallely, when there is no receiver ready the message gets stuck at the sender, but using buffered channels we add storage capacity to the channels. 

- The loop `for i := range c` receives values from the channel repeatedly until it is closed.
- Only the sender should close a channel, never the receiver. Sending on a closed channel will cause a panic.
- Channels aren't like files; you don't usually need to close them. Closing is only necessary when the receiver must be told there are no more values coming, such as to terminate a range loop.

- The select statement lets a goroutine wait on multiple communication operations.

- A select blocks until one of its cases can run, then it executes that case. It chooses one at random if multiple are ready.

    ```go
    func fibonacci(c, quit chan int) {
        x, y := 0, 1
        for {
            select {
            case c <- x:
                x, y = y, x+y
            case <-quit:
                fmt.Println("quit")
                return
            }
        }
    }
    func main() {
        c := make(chan int)
        quit := make(chan int)
        go func() {
            for i := 0; i < 10; i++ {
                fmt.Println(<-c)
            }
            quit <- 0
        }()
        fibonacci(c, quit)
    }
    ```

- Go can listen on multiple channels at once. And can respond if message comes in on any of them.

- The default case in a select is run if no other case is ready.

- Use a default case to try a send or receive without blocking:
    ```go
    select {
    case i := <-c:
        // use i
    default:
        // receiving from c would block
    }
    ```
    
- e.g.

    ```go
    func main() {
        tick := time.Tick(100 * time.Millisecond)
        boom := time.After(500 * time.Millisecond)
        for {
            select {
            case <-tick:
                fmt.Println("tick.")
            case <-boom:
                fmt.Println("BOOM!")
                return
            default:
                fmt.Println("    .")
                time.Sleep(50 * time.Millisecond)
            }
        }
    }
    ```

- Asynchronous Data transfer

    ```go
    func main() {
        a := []int{1, 2, 3, 4, 5}
        l := len(a)
	
        ch := make(chan int, l)
        for i := 0; i < l; i++ {
            ch <- a[i] * a[i]
        }
        close(ch)
	
        for d := range ch {
            print(d, " ")
        }
    }
    ```

### GOMAXPROCS

Why doesn't my multi-goroutine program use multiple CPUs?

The number of CPUs available simultaneously to executing goroutines is controlled by the `GOMAXPROCS` shell environment variable. In earlier releases of Go, the default value was 1, but as of Go 1.5 the default value is the number of cores available. Therefore programs compiled after 1.5 should demonstrate parallel execution of multiple goroutines. To change the behavior, set the environment variable or use the similarly-named function of the `runtime` package to configure the run-time support to utilize a different number of threads.

Programs that perform parallel computation might benefit from a further increase in `GOMAXPROCS`. However, be aware that concurrency is not parallelism.

Ref: 

   - Go FAQ: https://golang.org/doc/faq#Why_no_multi_CPU
   - Stack: http://stackoverflow.com/a/17868720

- e.g. Async data writing

    ```go
    func main() {
        runtime.GOMAXPROCS(4)
    
        f, _ := os.Create("./xyz")
        f.Close()
    
        log_chan := make(chan string, 100)
        go func() {
            for {
                msg, ok := <-log_chan
                if ok {
                    f, _ := os.OpenFile("./xyz", os.O_APPEND|os.O_WRONLY, 0600)
                    f.WriteString(msg)
                    f.Close()
                }
            }
        }()
        
        for i := 1; i < 10; i++ {
            for j := 1; j < 10; j++ {
                log_chan <- fmt.Sprintf("%d * %d = %d\n", i, j, i*j)
            }
        }
        fmt.Scanln() // added so that main waits for go-routines
    }

    ```